# Interpreting Seasons from dates

In [143]:
import datetime as dt
import numpy
import pandas as pd
import os
import os.path as p
import numpy as np
import shutil

In [21]:
def get_season(now):
    Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
    seasons = [('winter', (dt.date(Y,  1,  1),  dt.date(Y,  3, 20))),
           ('spring', (dt.date(Y,  3, 21),  dt.date(Y,  6, 20))),
           ('summer', (dt.date(Y,  6, 21),  dt.date(Y,  9, 22))),
           ('autumn', (dt.date(Y,  9, 23),  dt.date(Y, 12, 20))),
           ('winter', (dt.date(Y, 12, 21),  dt.date(Y, 12, 31)))]
    now = dt.datetime.strptime(str(now), '%Y%m%d').date()
    print(now)
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

In [22]:
get_season('20200101')

2020-01-01


'winter'

# Finding optimal frames

In [88]:
df = pd.read_csv('../data/Phoenix/temporal_references.csv')
# print(df)
target_dates = df['target_date'].values.tolist()
reference_dates = df['ref_date'].values.tolist()
target_dates = [dt.datetime.strptime(str(d), '%Y%m%d').date() for d in target_dates]
reference_dates = [dt.datetime.strptime(str(d), '%Y%m%d').date() for d in reference_dates]
days_delta, mod_days_delta = [], []
for x, y in zip(target_dates, reference_dates):
    days_delta.append(abs((x - y).days))
    y_same_year = y.replace(year=x.year)
    same_year_delta = abs((x - y_same_year).days)
    mod_days_delta.append(min(same_year_delta, 365 - same_year_delta))
df['mod_days_delta'] = mod_days_delta
df['days_delta'] = days_delta
df.to_csv('../data/Phoenix/temporal_references2.csv', index=False)

In [89]:
df = pd.read_csv('../data/Phoenix/temporal_references2.csv')\
.drop(['target_syn_cloud_date', 'target_synthetic_occlusion_percentage', 'MSE'], axis=1)
df = df.loc[df['reference_gt_occlusion_percentage'] < .1]
# print(df)
# print('-------------')
df = df.loc[df['mod_days_delta'] < 2 * 16 + 1]
# print(df)
# print('-------------')
df = df.sort_values(by=['days_delta'])
print(df)

     target_date  ref_date  reference_gt_occlusion_percentage       MAE  \
104     20220102  20220102                           0.001220  0.000000   
103     20220102  20211217                           0.003068  0.755963   
106     20220102  20220203                           0.005343  0.979223   
81      20220102  20210131                           0.001915  0.700218   
82      20220102  20210115                           0.014011  1.043159   
79      20220102  20201230                           0.005382  0.811635   
59      20220102  20200113                           0.004892  0.785347   
38      20220102  20190126                           0.004644  0.751778   
35      20220102  20181209                           0.002498  0.836874   
20      20220102  20180123                           0.004932  0.935939   
21      20220102  20171222                           0.005529  0.983291   
22      20220102  20171206                           0.006784  1.111147   
7       20220102  2017010

In [101]:
print(df['ref_date'].values)

[20220102 20211217 20220203 20210131 20210115 20201230 20200113 20190126
 20181209 20180123 20171222 20171206 20170104]


In [111]:
print(df.loc[df['ref_date'] == 20220102]['MAE'])

104    0.0
Name: MAE, dtype: float64


In [114]:
h = ''
print(f'sad{h}')

sad


# Average predictions

In [102]:
import cv2

In [104]:
img1 = cv2.imread('../data/Phoenix/bt_series/LC08_B10_20170104.tif', -1)
img2 = cv2.imread('../data/Phoenix/bt_series/LC08_B10_20170205.tif', -1)
print(img1.shape)
print(img2.shape)

(2471, 2626)
(2471, 2626)


In [106]:
avg_img = (img1 + img2) / 2
print(avg_img.shape)


(2471, 2626)


In [122]:
a = np.array([1, 2, 3, 4])
b = np.array([6, 6, 6, 6])
t = np.array([True, False, True, False])
r = np.zeros_like(a)
r[t] = a[t]
r[~t] = b[~t]
print(r)

[1 6 3 6]


# Managing Timelapse files

In [142]:
filepath = '../data/Houston/output/'
files = os.listdir(filepath)
files = [f for f in files if 'png' in f]
occluded_files = [f for f in files if 'occluded' in f]
spatial_files = [f for f in files if 'spatial' in f]
temporal_files = [f for f in files if 'temporal' in f]
st_files = [f for f in files if '_st' in f]
assert len(occluded_files) == len(st_files)
# print(occluded_files)

if not p.exists(p.join(filepath, 'occluded')):
    os.mkdir(p.join(filepath, 'occluded'))
    os.mkdir(p.join(filepath, 'spatial'))
    os.mkdir(p.join(filepath, 'temporal'))
    os.mkdir(p.join(filepath, 'st'))

In [146]:
for f in occluded_files:
    src = p.join(filepath, f)
    dst = p.join(filepath, 'occluded', f)
    shutil.copyfile(src, dst)

for f in spatial_files:
    src = p.join(filepath, f)
    dst = p.join(filepath, 'spatial', f)
    shutil.copyfile(src, dst)
    
for f in temporal_files:
    src = p.join(filepath, f)
    dst = p.join(filepath, 'temporal', f)
    shutil.copyfile(src, dst)
    
for f in st_files:
    src = p.join(filepath, f)
    dst = p.join(filepath, 'st', f)
    shutil.copyfile(src, dst)